In [23]:
from keras.models import Sequential, Model
from keras.layers import SimpleRNN,LSTM, Dense, Input, BatchNormalization
import numpy as np

In [29]:
import string
data = open("../data/alice_wonderland.txt",'r').read().lower()
vocab = list(set(data))
#len(vocab), len(data)
data = " ".join(data.split())

In [30]:
char_to_index = { ch:i for i,ch in enumerate(vocab)}
index_to_char = { i:ch for i,ch in enumerate(vocab)}

#char_to_index, index_to_char

In [31]:
sentence_length = 40

def change_data(data):
    #Encode string to numeric values
    encoded_data = [char_to_index[ch] for ch in data]
    end_limit = len(encoded_data) - sentence_length
    vocab_size = len(vocab)
    sentences = [] 
    Y = []
    for i in range(0,end_limit):
        # generate sample
        sentences.append(encoded_data[i:i+sentence_length]) 
        # generate output char
        output = np.zeros((vocab_size,1))
        output[encoded_data[i+sentence_length]] = True
        Y.append(output)
    # Now change sentences to one hot encoded data sequence
    X = np.zeros((len(sentences),sentence_length,vocab_size))
    for i in range(len(sentences)):
        for j in range(len(sentences[i])):
            X[i][j][sentences[i][j]] = True
    return X,Y

In [ ]:
X,Y = change_data(data) 

In [ ]:
train_size = int(len(X) * 0.67)
test_size = len(X) - train_size
trainX, testX = X[0:train_size,:,:], X[train_size: ,:,:]
trainY, testY = Y[0:train_size], Y[train_size:]

In [ ]:
testY = np.array(testY)
testY = np.reshape(testY, (testY.shape[0],testY.shape[1]))
trainY = np.array(trainY)
trainY = np.reshape(trainY, (trainY.shape[0],trainY.shape[1]))

In [24]:
lstm_output_size = 100
dropout_rate = 0

# Input Layer
ip = Input(shape=[sentence_length,len(vocab)])

# LSTM Layer 1
input_ = BatchNormalization(axis=-1)(ip)
lstm_out = SimpleRNN(units=lstm_output_size, return_sequences=True, dropout=dropout_rate, recurrent_dropout=dropout_rate)(input_)
input_ = lstm_out

# # LSTM Layer 2
# input_ = BatchNormalization(axis=-1)(input_)
# lstm_out = RNN(units=lstm_output_size, return_sequences=True, dropout=dropout_rate, recurrent_dropout=dropout_rate)(input_)
# input_ = lstm_out

# # LSTM Layer 3
# input_ = BatchNormalization(axis=-1)(input_)
# lstm_out = LSTM(units=lstm_output_size, return_sequences=True, dropout=dropout_rate, recurrent_dropout=dropout_rate)(input_)
# input_ = lstm_out

# LSTM Layer 4
input_ = BatchNormalization(axis=-1)(input_)
lstm_out = SimpleRNN(units=lstm_output_size, return_sequences=False, dropout=dropout_rate, recurrent_dropout=dropout_rate)(input_)
input_ = lstm_out

# Output layer
output = Dense(len(vocab), activation='softmax')(lstm_out)

# Definingn Model
model = Model(inputs=[ip],outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 40, 41)            0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 40, 41)            164       
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 40, 100)           14200     
_________________________________________________________________
batch_normalization_10 (Batc (None, 40, 100)           400       
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 100)               20100     
_________________________________________________________________
dense_3 (Dense)              (None, 41)                4141      
Total params: 39,005
Trainable params: 38,723
Non-trainable params: 282
_________________________________________________________________


In [25]:
model.fit(trainX,trainY,epochs=5,batch_size=50)

Epoch 1/5
21657/21657 [==============================] - 36s 2ms/step - loss: 2.4690 - acc: 0.3310
Epoch 2/5
21657/21657 [==============================] - 32s 1ms/step - loss: 1.9613 - acc: 0.4304
Epoch 3/5
21657/21657 [==============================] - 38s 2ms/step - loss: 1.8178 - acc: 0.4648
Epoch 4/5
21657/21657 [==============================] - 38s 2ms/step - loss: 1.7205 - acc: 0.4886
Epoch 5/5
21657/21657 [==============================] - 32s 1ms/step - loss: 1.6403 - acc: 0.5083


In [26]:
model.evaluate(testX,testY)

10667/10667 [==============================] - 8s 780us/step


[1.8731717939976733, 0.4490484672493683]

In [27]:
seed_sentence = data[0:sentence_length+1] # Must be of size sentence_lenth + 1
seed_sentence

"the adventure of the three gables i don't"

In [28]:
current_sentence = seed_sentence
output = current_sentence
for i in range(100):
    currentX,_ = change_data(current_sentence)
    currentY = model.predict(currentX)
    predicted_char = index_to_char[np.argmax(currentY)]
    current_sentence = current_sentence[1:] + predicted_char
    output = output + predicted_char
output

"the adventure of the three gables i don'tt  aonnee  thhee  wiitn  hheadr ywoarl ywoarl ywoarl ywoarl ywoarl ywoarl ywoarl ywoarl ywoarl ywoar"

In [13]:
#model.save('test_model.h5')